In [ ]:
# Instalamos los paquetes necesarios 
!pip install -U transformers datasets
!pip install sklearn
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.3 MB/s 
     |████████████████████████████████| 362 kB 50.5 MB/s 
     |████████████████████████████████| 596 kB 55.4 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.6 MB/s 
     |████████████████████████████████| 140 kB 44.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 212 kB 52.7 MB/s 
     |████████████████████████████████| 127 kB 52.2 MB/s 
     |████████████████████████████████| 271 kB 31.8 MB/s 
     |████████████████████████████████| 94 kB 1.8 MB/s 
     |████████████████████████████████| 144 kB 47.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing

In [ ]:
import pandas as pd 

train_df = pd.read_csv('./train.csv', encoding='utf-8')
test_df = pd.read_csv('./test.csv', encoding='utf-8')

In [ ]:
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


## Data Processing 

In [ ]:
import re
import unicodedata
import string

url = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
url2= '(www\.)(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
emoticons= re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         u"\U0001f926-\U0001f937"
         u"\U00010000-\U0010ffff"
         u"\u2640-\u2642"
         u"\u2600-\u2B55"
         u"\u200d"
         u"\u23cf"
         u"\u23e9"
         u"\u231a"
         u"\ufe0f"  # dingbats
         u"\u3030"
         "]+", flags=re.UNICODE)
patron = re.compile('#|@|'+'|'+url+'|'+url2)

def clean_tweet(tweet): 
  # Eliminas los URLs, menciones y hashtags
  tweet_clean = patron.sub('',tweet)
  # Eliminamos los emoticons 
  tweet_clean = emoticons.sub('',tweet_clean)
  # Eliminamos las puntaciones
  tweet_clean = "".join([i for i in tweet_clean if i not in string.punctuation])
  return tweet_clean.rstrip()

In [ ]:
train_df['tweet_clean'] = train_df['text'].apply(lambda x:clean_tweet(x))
test_df['tweet_clean'] = test_df['text'].apply(lambda x:clean_tweet(x))

### Split the train data for validation

In [ ]:
p_train = 0.80 
p_eval = 0.20 

from sklearn.model_selection import train_test_split 

train_df = train_df.sample(frac=1)
train_df, eval_df = train_test_split (train_df, test_size = p_eval)
print("Examples for training: ", len(train_df))
print("Examples for training: ", len(eval_df))

Examples for training:  6090
Examples for training:  1523


In [ ]:
print("Examples with 1: ", len(train_df.loc[train_df['target'] == 1]))
print("Examples with 0 : ", len(train_df.loc[train_df['target'] == 0]))

Examples with 1:  2627
Examples with 0 :  3463


In [ ]:
import torch 
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import tensorflow as tf

# Roberta model 
path_bert_model = 'bert-base-uncased'
NUM_LABEL = 2
# Obtenemos el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained (path_bert_model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
# Train
train_encodings = tokenizer(train_df['tweet_clean'].tolist (), truncation=True, padding=True, return_tensors='tf')
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_df['target']))

# Eval 
eval_encodings = tokenizer(eval_df['tweet_clean'].tolist (), truncation=True, padding=True, return_tensors='tf')
eval_dataset = tf.data.Dataset.from_tensor_slices((dict(eval_encodings), eval_df['target']))


## Compile and fit

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

roberta_model = TFAutoModelForSequenceClassification.from_pretrained (path_bert_model, num_labels=NUM_LABEL)

roberta_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

EPOCHS = 8
BATCH_SIZE = 16
roberta_model.fit(train_dataset.shuffle(100).batch(BATCH_SIZE),
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_data=eval_dataset.shuffle(100).batch(BATCH_SIZE),
)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/8
381/381 [==============================] - 105s 219ms/step - loss: 0.4443 - sparse_categorical_accuracy: 0.8034 - val_loss: 0.4030 - val_sparse_categorical_accuracy: 0.8365
Epoch 2/8
381/381 [==============================] - 83s 219ms/step - loss: 0.2978 - sparse_categorical_accuracy: 0.8846 - val_loss: 0.4396 - val_sparse_categorical_accuracy: 0.8339
Epoch 3/8
381/381 [==============================] - 83s 219ms/step - loss: 0.2029 - sparse_categorical_accuracy: 0.9274 - val_loss: 0.6130 - val_sparse_categorical_accuracy: 0.8286
Epoch 4/8
381/381 [==============================] - 83s 218ms/step - loss: 0.1327 - sparse_categorical_accuracy: 0.9527 - val_loss: 0.7417 - val_sparse_categorical_accuracy: 0.8214
Epoch 5/8
381/381 [==============================] - 83s 219ms/step - loss: 0.1004 - sparse_categorical_accuracy: 0.9617 - val_loss: 0.7688 - val_sparse_categorical_accuracy: 0.8116
Epoch 6/8
381/381 [==============================] - 83s 218ms/step - loss: 0.0812 - spar

## Predictions

In [ ]:
# Save the model and valid with test data 

roberta_model.save_pretrained('clasificationModel')
tokenizer.save_pretrained('clasificationModel')

('clasificationModel/tokenizer_config.json',
 'clasificationModel/special_tokens_map.json',
 'clasificationModel/vocab.txt',
 'clasificationModel/added_tokens.json',
 'clasificationModel/tokenizer.json')

In [ ]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained('clasificationModel')

# Imprimimos las predicciones obtenidas
result = []
for text in test_df['tweet_clean']:
  predict_input = tokenizer.encode (text, truncation=True, padding=True, return_tensors="tf")
  tf_output = tf_model.predict(predict_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1).numpy()[0]
  if tf_prediction[0] > tf_prediction[1]:
    result.append(0)
  else: 
    result.append(1)

Some layers from the model checkpoint at clasificationModel were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at clasificationModel.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
dfsub = pd.DataFrame({'id' : test_df['id'].to_list(),'target': result})
dfsub.to_csv('submission.csv', index=False)
dfsub = pd.read_csv('submission.csv')
dfsub

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,1
3259,10865,1
3260,10868,1
3261,10874,1
